# setup

In [40]:
# !pip install -r ../requirements.txt
# !conda install cudatoolkit -y
# !pip install cuda-python>=12.3
!cp ~/.cache/.netrc ~

In [2]:
import os

if not os.path.exists("../scripts/speech_to_text_hybrid_rnnt_ctc_bpe.py"):
    !wget -P ../scripts https://raw.githubusercontent.com/NVIDIA/NeMo/refs/heads/main/examples/asr/asr_hybrid_transducer_ctc/speech_to_text_hybrid_rnnt_ctc_bpe.py

if not os.path.exists("../configs/fastconformer_hybrid_transducer_ctc_bpe.yaml"):
    !wget -P ../configs https://raw.githubusercontent.com/NVIDIA/NeMo/refs/heads/main/examples/asr/conf/fastconformer/hybrid_transducer_ctc/fastconformer_hybrid_transducer_ctc_bpe.yaml

if not os.path.exists("../scripts/process_asr_text_tokenizer.py"):
    !wget -P ../scripts https://raw.githubusercontent.com/NVIDIA/NeMo/refs/heads/main/scripts/tokenizers/process_asr_text_tokenizer.py

if not os.path.exists("../scripts/convert_hf_dataset_to_nemo.py"):
    !wget -P ../scripts https://raw.githubusercontent.com/NVIDIA/NeMo/refs/heads/main/scripts/speech_recognition/convert_hf_dataset_to_nemo.py

In [3]:
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_fa_fastconformer_hybrid_large")
cfg = asr_model.cfg
OmegaConf.save(cfg, "../configs/fastconformer_hybrid_transducer_ctc_bpe_persian_pretrained.yaml")

stt_fa_fastconformer_hybrid_large.nemo:   0%|          | 0.00/459M [00:00<?, ?B/s]

[NeMo I 2025-07-20 05:27:42 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-20 05:27:43 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: dummy
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    max_duration: 10
    min_duration: 0.5
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    use_lhotse: true
    lhotse:
      shar_path: /data_artifacts/data/shar/train
      batch_duration: 1200
      quadratic_duration: 15
      num_buckets: 10
      num_cuts_for_bins_estimate: 10000
      buffer_size: 10000
      shuffle_buffer_size: 10000
    
[NeMo W 2025-07-20 05:27:43 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and prov

[NeMo I 2025-07-20 05:27:43 nemo_logging:393] PADDING: 0
[NeMo I 2025-07-20 05:27:43 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-20 05:27:43 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-07-20 05:27:43 nemo_logging:405] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-07-20 05:27:44 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-07-20 05:27:44 nemo_logging:405] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-07-20 05:27:44 nemo_logging:393] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /home/jovyan/.cache/huggingface/hub/models--nvidia--stt_fa_fastconformer_hybrid_large/snapshots/249cf5bf70dda7220a60ddeeecff2f6aad8e1784/stt_fa_fastconformer_hybrid_large.nemo.


In [2]:
# python [NEMO_GIT_FOLDER]/scripts/tokenizers/process_asr_text_tokenizer.py \
#   --manifest="train_manifest.json" \
#   --data_root="<OUTPUT DIRECTORY FOR TOKENIZER>" \
#   --vocab_size=1024 \
#   --tokenizer="spe" \
#   --spe_type="unigram" \
#   --spe_character_coverage=1.0 \
#   --no_lower_case \
#   --log

# dataset preparation

In [2]:
import os
import sys_append
from tqdm import tqdm
from utils.create_dataset import convert_hf_dataset_nemo
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest

In [ ]:
convert_hf_dataset_nemo(dataset_name="hsekhalilian/sorted_commonvoice", split="train")
convert_hf_dataset_nemo(dataset_name="hsekhalilian/sorted_commonvoice", split="dev")
convert_hf_dataset_nemo(dataset_name="hsekhalilian/sorted_commonvoice", split="test")

In [3]:
manifest_dir = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests"

train_manifest = f"{manifest_dir}/train_manifest.json"
dev_manifest = f"{manifest_dir}/dev_manifest.json"
test_manifest = f"{manifest_dir}/test_manifest.json"

In [4]:
# Processing pipeline
def apply_preprocessors(manifest_data, preprocessors):
    for processor in preprocessors:
        for idx in tqdm(range(len(manifest_data)), desc=f"Applying {processor.__name__}"):
            manifest_data[idx] = processor(manifest_data[idx])

    print("Finished processing manifest !")
    return manifest_data

def write_processed_manifest(manifest_data, original_path):
    original_manifest_name = os.path.basename(original_path)
    new_manifest_name = original_manifest_name.replace(".json", "_processed.json")

    manifest_dir = os.path.split(original_path)[0]
    filepath = os.path.join(manifest_dir, new_manifest_name)
    write_manifest(filepath, manifest_data)
    print(f"Finished writing manifest: {filepath}")
    
    return filepath

In [5]:
def make_lower_case(sample):
    sample["text"] = sample["text"].lower().strip()
    return sample

def do_nothing(sample):
    return sample
    
PREPROCESSORS = [
    make_lower_case,
]

In [6]:
# Load manifests
train_manifest_data = read_manifest(train_manifest)
dev_manifest_data = read_manifest(dev_manifest)
test_manifest_data = read_manifest(test_manifest)

# Apply preprocessing
train_data_processed = apply_preprocessors(train_manifest_data, PREPROCESSORS)
dev_data_processed = apply_preprocessors(dev_manifest_data, PREPROCESSORS)
test_data_processed = apply_preprocessors(test_manifest_data, PREPROCESSORS)

# Write new manifests
train_manifest_cleaned = write_processed_manifest(train_data_processed, train_manifest)
dev_manifest_cleaned = write_processed_manifest(dev_data_processed, dev_manifest)
test_manifest_cleaned = write_processed_manifest(test_data_processed, test_manifest)

Applying make_lower_case: 100%|██████████| 30061/30061 [00:00<00:00, 2224270.06it/s]


Finished processing manifest !


Applying make_lower_case: 100%|██████████| 10540/10540 [00:00<00:00, 2092783.76it/s]


Finished processing manifest !


Applying make_lower_case: 100%|██████████| 10540/10540 [00:00<00:00, 2271735.05it/s]


Finished processing manifest !
Finished writing manifest: /home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/train_manifest_processed.json
Finished writing manifest: /home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/dev_manifest_processed.json
Finished writing manifest: /home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/test_manifest_processed.json


## create tokenizer

In [23]:
VOCAB_SIZE = 1024
TOKENIZER_TYPE = "bpe"
tokenizer_dir = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/tokenizers"

!python ../scripts/process_asr_text_tokenizer.py \
  --manifest=$train_manifest_cleaned,$dev_manifest_cleaned \
  --vocab_size=$VOCAB_SIZE \
  --data_root=$tokenizer_dir \
  --tokenizer="spe" \
  --spe_type=$TOKENIZER_TYPE \
  --spe_character_coverage=1.0 \
  --log

usage: process_asr_text_tokenizer.py [-h]
                                     (--manifest MANIFEST | --data_file DATA_FILE)
                                     --data_root DATA_ROOT
                                     [--vocab_size VOCAB_SIZE]
                                     [--tokenizer {spe,wpe}]
                                     [--spe_type {bpe,unigram,char,word}]
                                     [--spe_character_coverage SPE_CHARACTER_COVERAGE]
                                     [--spe_bos] [--spe_eos] [--spe_pad]
                                     [--spe_user_defined_symbols SPE_USER_DEFINED_SYMBOLS [SPE_USER_DEFINED_SYMBOLS ...]]
                                     [--spe_control_symbols SPE_CONTROL_SYMBOLS [SPE_CONTROL_SYMBOLS ...]]
                                     [--spe_split_digits]
                                     [--spe_remove_extra_whitespaces]
                                     [--spe_sample_size SPE_SAMPLE_SIZE]
                            

In [76]:
TOKENIZER_DIR = f"{tokenizer_dir}/tokenizer_spe_{TOKENIZER_TYPE}_v{VOCAB_SIZE}/"
print("Tokenizer directory :", TOKENIZER_DIR)

with open(os.path.join(TOKENIZER_DIR, 'tokenizer.vocab')) as f:
      tokens = f.readlines()

num_tokens = len(tokens)
print("Number of tokens : ", num_tokens)

with open(os.path.join(TOKENIZER_DIR, 'vocab.txt')) as f:
      vocab = f.read()
print(vocab.split())

Tokenizer directory : /home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/tokenizers/tokenizer_spe_bpe_v1024/
Number of tokens :  1024
['ب', 'ا', 'م', 'د', 'ک', '##را', '##ان', '##ار', '##ست', 'ت', 'خ', '##ای', '##ند', 'ن', '##رد', 'ش', 'می', 'و', 'پ', '##ین', 'را', 'س', 'به', '##ود', 'ه', 'ر', 'او', '##ید', '##اد', 'در', 'آ', 'با', 'چ', 'ج', 'از', 'است', 'ز', '##های', 'کرد', '##ام', '##یم', 'این', 'گ', '##ال', 'ح', 'خو', '##اه', '##یر', 'ی', 'که', '##ور', 'ف', '##شت', 'من', '##ون', '##فت', '##هی', 'رو', 'ق', '##لی', 'شد', '##ها', 'بر', 'ع', '##اب', 'بود', 'تو', 'هم', 'کن', '##ری', 'دار', 'خود', 'برا', '##از', '##خت', '##می', 'یک', '##سی', 'دو', '##رو', '##ات', 'ص', 'آن', '##نگ', 'بی', 'ل', 'میک', '##وی', '##اری', 'برای', '##لا', '##ران', '##مان', 'سر', 'ما', 'مو', '##ما', '##اشت', 'پی', '##ده', '##وان', '##ول', '##زی', 'داد', '##گی', 'کار', 'نی', '##تر', '##بی', '##تی', '##انه', 'دی', '##نی', 'خواه', 'تا', 'کردن', '##انی', '##در', 'پر', '##اس', '##گر', '##وش', 'چه', 'مح', '

# load english pretrained model

In [1]:
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models import EncDecHybridRNNTCTCBPEModel
from nemo.utils import logging, exp_manager

In [2]:
model_name = "stt_en_fastconformer_hybrid_large_pc"
asr_model = EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name=model_name)

TOKENIZER_DIR = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/tokenizers/tokenizer_spe_bpe_v1024/"
pretrained_decoder = asr_model.decoder.state_dict()
asr_model.cfg.labels = None
asr_model.change_vocabulary(new_tokenizer_dir=TOKENIZER_DIR, new_tokenizer_type="bpe")

# Insert preserved model weights if shapes match
if list(asr_model.decoder.state_dict().values())[0].shape == list(pretrained_decoder.values())[0].shape:
    asr_model.decoder.load_state_dict(pretrained_decoder)
    logging.info("Decoder shapes matched - restored weights from pre-trained model")
else:
    logging.info("\nDecoder shapes did not match - could not restore decoder weights from pre-trained model.")

[NeMo I 2025-07-20 09:48:29 nemo_logging:393] Found existing object /home/jovyan/.cache/torch/NeMo/NeMo_2.3.2/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo.
[NeMo I 2025-07-20 09:48:29 nemo_logging:393] Re-using file from: /home/jovyan/.cache/torch/NeMo/NeMo_2.3.2/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo
[NeMo I 2025-07-20 09:48:29 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-07-20 09:48:29 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-20 09:48:30 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    is_concat: false
    concat_sampling_technique: random
    concat_sampling_temperature: 8
    concat_shuffle: true
    concat_sampling_scale: 1.0
    manifest_filepath:
    - - /data/mml/en/tarred_train/pcstrip_sharded_manifests/manifest__OP_0..511_CL_.json
    tarred_audio_filepaths:
    - - /data/mml/en/tarred_train/audio__OP_0..511_CL_.tar
    concat_sampling_probabilities:
    - 1.0
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: true
    shard_manifests: true
    defer_setup: true
    shuffle_n: 528387
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo

[NeMo I 2025-07-20 09:48:30 nemo_logging:393] PADDING: 0
[NeMo I 2025-07-20 09:48:31 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-20 09:48:31 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-20 09:48:31 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-20 09:48:32 nemo_logging:393] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /home/jovyan/.cache/torch/NeMo/NeMo_2.3.2/stt_en_fastconformer_hybrid_large_pc/465b32000fc320f5905fda11a1866ef6/stt_en_fastconformer_hybrid_large_pc.nemo.


[NeMo W 2025-07-20 09:48:32 nemo_logging:405] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2025-07-20 09:48:32 nemo_logging:405] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2025-07-20 09:48:32 nemo_logging:405] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2025-07-20 09:48:32 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-07-20 09:48:32 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-20 09:48:32 nemo_logging:393] Changed tokenizer of the RNNT decoder to ['<unk>', '▁ب', '▁ا', '▁م', '▁د', '▁ک', 'را', 'ان', 'ار', 'ست', '▁ت', '▁خ', 'ای', 'ند', '▁ن', 'رد', '▁ش', '▁می', '▁و', '▁پ', 'ین', '▁را', '▁س', '▁به', 'ود', '▁ه', '▁ر', '▁او', 'ید', 'اد', '▁در', '▁آ', '▁با', '▁چ', '▁ج', '▁از', '▁است', '▁ز', 'های', '▁کرد', 'ام', 'یم', '▁این', '▁گ', 'ال', '▁ح', '▁خو', 'اه', 'یر', '▁ی', '▁که', 'ور', '▁ف', 'شت', '▁من', 'ون', 'فت', 'هی', '▁رو', '▁ق', 'لی', '▁شد', 'ها', '▁بر', '▁ع', 'اب', '▁بود', '▁تو', '▁هم', '▁کن', 'ری', '▁دار', '▁خود', '▁برا', 'از', 'خت', 'می', '▁یک', 'سی', '▁دو', 'رو', 'ات', '▁ص', '▁آن', 'نگ', '▁بی', '▁ل', '▁میک', 'وی', 'اری', '▁برای', 'لا', 'ران', 'مان', '▁سر', '▁ما', '▁مو', 'ما', 'اشت', '▁پی', 'د

In [3]:
import torch
import torch.nn as nn

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)
     


freeze_encoder = True
freeze_encoder = bool(freeze_encoder)

if freeze_encoder:
  asr_model.encoder.freeze()
  asr_model.encoder.apply(enable_bn_se)
  logging.info("Model encoder has been frozen")
else:
  asr_model.encoder.unfreeze()
  logging.info("Model encoder has been un-frozen")

[NeMo I 2025-07-20 09:48:33 nemo_logging:393] Model encoder has been frozen


# update configs

In [4]:
cfg = OmegaConf.load("../configs/fastconformer_hybrid_transducer_ctc_bpe.yaml")

cfg.exp_manager.create_tensorboard_logger = False
cfg.exp_manager.create_wandb_logger = True
cfg.exp_manager.wandb_logger_kwargs = {"name": "nemo_fastconformer", "project": "persian-asr"}

cfg.trainer.log_every_n_steps = 50

In [17]:
train_manifest_cleaned = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/train_manifest_processed.json"
dev_manifest_cleaned = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/dev_manifest_processed.json"
test_manifest_cleaned = "/home/jovyan/.cache/datasets/hsekhalilian___sorted_commonvoice/manifests/test_manifest_processed.json"

with open_dict(cfg):
    cfg.model.tokenizer["dir"] = TOKENIZER_DIR
    
    # Train dataset
    cfg.model.train_ds.manifest_filepath = train_manifest_cleaned
    cfg.model.train_ds.batch_size = 64
    cfg.model.train_ds.num_workers = 1
    
    # Validation dataset
    cfg.model.validation_ds.manifest_filepath = dev_manifest_cleaned
    cfg.model.validation_ds.batch_size = 8
    cfg.model.validation_ds.num_workers = 1
    
    # Test dataset
    cfg.model.test_ds.manifest_filepath = test_manifest_cleaned
    cfg.model.test_ds.batch_size = 8
    cfg.model.test_ds.num_workers = 1

cfg = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))

asr_model.setup_training_data(cfg.model.train_ds)
asr_model.setup_multiple_validation_data(cfg.model.validation_ds)
asr_model.setup_multiple_test_data(cfg.model.test_ds)
asr_model.cfg.tokenizer = cfg.model.tokenizer

[NeMo I 2025-07-20 09:49:29 nemo_logging:393] Dataset loaded with 30061 files totalling 34.08 hours
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] 0 files were filtered totalling 0.00 hours
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] Dataset loaded with 10540 files totalling 12.67 hours
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] 0 files were filtered totalling 0.00 hours
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] Dataset loaded with 10540 files totalling 12.09 hours
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] 0 files were filtered totalling 0.00 hours


In [18]:
import lightning.pytorch as pl
from nemo.utils.exp_manager import exp_manager
from nemo.utils.trainer_utils import resolve_trainer_cfg

cfg.trainer.strategy = "auto"
trainer = pl.Trainer(**resolve_trainer_cfg(cfg.trainer))

exp_manager(trainer, cfg.get("exp_manager", None))
asr_model.set_trainer(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2025-07-20 09:49:29 nemo_logging:393] ExpManager schema
[NeMo I 2025-07-20 09:49:29 nemo_logging:393] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'lo

In [19]:
print(OmegaConf.to_yaml(cfg.model.optim))
print(OmegaConf.to_yaml(cfg.model.spec_augment))

name: adamw
lr: 5.0
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: NoamAnnealing
  d_model: 512
  warmup_steps: 10000
  warmup_ratio: null
  min_lr: 1.0e-06

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
freq_masks: 2
time_masks: 10
freq_width: 27
time_width: 0.05



In [ ]:
trainer.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2025-07-20 09:49:30 nemo_logging:393] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        decoupled_weight_decay: True
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0005
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2025-07-20 09:49:30 nemo_logging:393] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7139b5c6e090>" 
    will be used during training (effective maximum steps = 470000) - 
    Parameters : 
    (warmup_steps: 1000
    warmup_ratio: null
    min_lr: 5.0e-05
    max_steps: 470000
    )



  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoder                  | 108 M  | eval 
2 | spec_augmentation | SpectrogramAugmentation           | 0      | train
3 | wer               | WER                               | 0      | train
4 | ctc_wer           | WER                               | 0      | train
5 | joint             | RNNTJoint                         | 1.4 M  | train
6 | decoder           | RNNTDecoder                       | 3.9 M  | train
7 | loss              | RNNTLoss                          | 0      | train
8 | ctc_decoder       | ConvASRDecoder                    | 525 K  | train
9 | ctc_loss          | CTCLoss                           | 0      | train
--------------------------------------------------------------------------------
5.9 M     Tr

Training: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2025-07-20 09:49:30 nemo_logging:393] Preemption requires torch distributed to be initialized, disabling preemption
[NeMo I 2025-07-20 09:49:30 nemo_logging:393] Disabled CUDA graphs for module <class 'nemo.collections.asr.models.hybrid_rnnt_ctc_bpe_models.EncDecHybridRNNTCTCBPEModel'>.decoding.decoding
[NeMo I 2025-07-20 09:49:30 nemo_logging:393] Disabled CUDA graphs for module <class 'nemo.collections.asr.metrics.wer.WER'>wer.decoding.decoding


[NeMo W 2025-07-20 09:49:30 nemo_logging:405] /opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
      warn(NumbaPerformanceWarning(msg))
    
[NeMo W 2025-07-20 09:49:30 nemo_logging:405] /opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
      warn(NumbaPerformanceWarning(msg))
    


[NeMo I 2025-07-20 09:49:48 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] reference:سبک تند و تیز نگارش او
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] predicted:گار
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] reference:خیار را معمولا در سالاد مصرف میکنند.
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] predicted:کامکارودزار
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] reference:پولها را روی میز پراند.
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] predicted:ذ کنی میگ
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] reference:عادت مذموم
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] predicted:تتت؟ تک
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] reference:مسئله یه چیز دیگه هست
[NeMo I 2025-07-20 09:49:48 nemo_logging:393] predicted:متن
[NeMo I 2025-07-20 09:49:

[NeMo W 2025-07-20 09:49:49 nemo_logging:405] CTC decoding strategy 'greedy' is slower than 'greedy_batch', which implements the same exact interface. Consider changing your strategy to 'greedy_batch' for a free performance improvement.


[NeMo I 2025-07-20 09:49:49 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:49:49 nemo_logging:393] reference:سبک تند و تیز نگارش او
[NeMo I 2025-07-20 09:49:49 nemo_logging:393] predicted:دریرد چهار مرتندگان مرتضیعیت ق همهروع خون چشماندهت مشک همه جه بیرون ام اینجا کلی جهمیز اگه چشمان مس اگه چشمان
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] reference:تمام عمرم را منتظر ماندم تا پدرم از من عذرخواهی کند
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] predicted:
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] reference:وقتی امپراتوری روم به سمت جنوب در حال گسترش بود
[NeMo I 2025-07-20 09:50:07 nemo_logging:393] predicted:
[NeMo I 2025-07-20 09:50:08 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:50:08 nemo_logging:393] reference:من دوقلو دارم.
[NeMo I 2025-07-20 09:50:08 nemo_logging:393] predicted:
[NeMo I 2025-07-20 09:50:08 nemo_logging:393] 
    
[NeMo I 2025-07-20 09:50:08 nemo_logging:

# test